# Langfuse 프롬프트 관리


---

## 1. Langfuse 소개

### 🔍 Langfuse란?
**Langfuse**는 LLM(Large Language Model) 애플리케이션을 위한 오픈소스 엔지니어링 플랫폼입니다.

#### 주요 기능
- **📊 관찰가능성(Observability)**: LLM 호출과 애플리케이션 로직 추적
- **📝 프롬프트 관리**: 버전 관리, 협업, 배포
- **📈 분석 및 메트릭**: 성능, 비용, 품질 지표
- **🔄 평가 및 실험**: A/B 테스트, 자동 평가

#### 최신 SDK 버전 정보 
- **v3 SDK (권장)**: OpenTelemetry 기반, 2025년 6월 정식 출시
- **v2 SDK**: 기존 버전, 계속 지원되지만 v3로 마이그레이션 권장

---

## 2. 환경 설정

### 📦 필수 라이브러리 설치

```bash
# 최신 버전 설치
pip install langfuse>=3.2.1 langchain>=0.1.10 langchain-openai python-dotenv
```

### 🔐 환경 변수 설정

```python
# .env 파일 생성
LANGFUSE_PUBLIC_KEY="pk-lf-..."
LANGFUSE_SECRET_KEY="sk-lf-..."
LANGFUSE_HOST="https://cloud.langfuse.com"  # EU 지역
# LANGFUSE_HOST="https://us.cloud.langfuse.com"  # US 지역

# OpenAI API 키 (선택사항)
OPENAI_API_KEY="sk-proj-..."
```

### ⚙️ 기본 설정 코드


`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
import warnings
from pprint import pprint
import json

warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [ ]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) Langfuse 클라이언트 설정`

In [ ]:
from langfuse import Langfuse, get_client

# Langfuse 클라이언트 초기화 
langfuse_client = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")
)

# 싱글톤 클라이언트 인스턴스 가져오기 
langfuse = get_client()


---

## 3. 프롬프트 관리 기초

### 📝 프롬프트 관리의 중요성

프롬프트 관리는 LLM 애플리케이션에서 핵심적인 요소입니다:

- **버전 관리**: 프롬프트 변경 내역 추적
- **코드 분리**: 프롬프트를 코드에서 분리하여 독립적 관리
- **협업**: 팀원 간 프롬프트 공유 및 수정
- **배포**: 환경별 프롬프트 배포 (개발/스테이징/프로덕션)
- **성능 모니터링**: 프롬프트 버전별 성능 비교

### 📋 프롬프트 타입

Langfuse는 두 가지 프롬프트 타입을 지원합니다:

1. **텍스트 프롬프트**: 단순 문자열 형태
2. **채팅 프롬프트**: 메시지 배열 형태 (role, content)

### 🎯 프롬프트 생성

In [ ]:
# 텍스트 프롬프트 생성
response = langfuse.create_prompt(
    name="movie-critic",           # 프롬프트 식별자
    type="text",                   # 프롬프트 타입
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],         # 환경 라벨
    tags=["movie", "qa", "text"],  # 분류 태그
    config={                       # 모델 설정
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 1000
    }
)

print(f"✅ 텍스트 프롬프트 생성 완료: {response.name}")

In [ ]:
# 채팅 프롬프트 생성
response = langfuse.create_prompt(
    name="movie-critic-chat",      # 프롬프트 식별자
    type="chat",                   # 채팅 타입
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 객관적이고 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user", 
            "content": "영화 {{movie}}에 대한 평가를 부탁드립니다."
        }
    ],
    labels=["staging"],            # 스테이징 환경
    tags=["movie", "qa", "chat"],  # 분류 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 1500
    }
)

print(f"✅ 채팅 프롬프트 생성 완료: {response.name}")

### 🏷️ 라벨 시스템

라벨은 프롬프트 버전을 관리하는 핵심 메커니즘입니다:

- **`production`**: 프로덕션 환경용 (기본값)
- **`staging`**: 스테이징 환경용
- **`latest`**: 가장 최근 버전
- **사용자 정의**: 예) `experiment-a`, `tenant-1`

---

## 4. 프롬프트 활용

### 📥 프롬프트 불러오기

In [ ]:
# 프로덕션 버전 가져오기 (기본값)
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 정보 출력
print(f"📋 프롬프트 이름: {prompt.name}")
print(f"🏷️ 라벨: {prompt.labels}")
print(f"🏷️ 태그: {prompt.tags}")
print(f"⚙️ 모델: {prompt.config.get('model', 'N/A')}")
print(f"🌡️ 온도: {prompt.config.get('temperature', 'N/A')}")
print(f"📝 프롬프트: {prompt.prompt}")
print("-" * 80)

In [ ]:
# LangChain용 프롬프트 변환
langchain_prompt_text = prompt.get_langchain_prompt()
print(f"🔗 LangChain 프롬프트: {langchain_prompt_text}")

In [ ]:
# 특정 버전으로 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)

# 특정 라벨로 가져오기
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="staging")
prompt_latest = langfuse.get_prompt("movie-critic-chat", label="latest")

print(f"📋 버전 1 프롬프트: {prompt_v1.prompt}")
print(f"📋 스테이징 프롬프트: {prompt_staging.prompt}")

### 🔄 프롬프트 버전 관리

In [ ]:
# 새 버전 생성 (같은 이름으로)
updated_response = langfuse.create_prompt(
    name="movie-critic",  # 동일한 이름 사용
    type="text",
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}에 대해 다음 관점에서 분석해주세요:\n1. 스토리\n2. 연출\n3. 연기\n4. 전체적 평가",
    labels=["production"],  # 프로덕션으로 업데이트
    tags=["movie", "qa", "updated"],
    config={
        "model": "gpt-4.1",  # 모델 업그레이드
        "temperature": 0.5  # 온도 조정
    }
)

print(f"✅ 프롬프트 업데이트 완료: v{updated_response.version}")

### 💾 프롬프트 캐싱

Langfuse SDK는 자동 캐싱을 제공합니다:

- **기본 TTL**: 60초
- **백그라운드 갱신**: 만료 시 백그라운드에서 새로고침
- **폴백 지원**: 네트워크 오류 시 캐시된 버전 사용

In [ ]:
# 캐싱은 자동으로 동작하며, 별도 설정 불필요
# 첫 번째 호출: API에서 가져오기
import time
start_time = time.time()
prompt1 = langfuse.get_prompt("movie-critic")
end_time = time.time()
print(f"⏱️ 첫 번째 호출 시간: {end_time - start_time:.2f}초")

# 두 번째 호출: 캐시에서 가져오기 (빠름)
start_time = time.time()
prompt2 = langfuse.get_prompt("movie-critic")
end_time = time.time()
print(f"⏱️ 두 번째 호출 시간: {end_time - start_time:.2f}초")


---

## 5. LangChain 통합

### 🔗 LangChain과 연동

Langfuse는 LangChain과 네이티브 통합을 제공합니다.

#### 텍스트 프롬프트와 LangChain 연동

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Langfuse에서 프롬프트 가져오기
prompt = langfuse.get_prompt("movie-critic")

# LangChain PromptTemplate 생성
langchain_prompt = PromptTemplate.from_template(
    prompt.get_langchain_prompt(),
    metadata={"langfuse_prompt": prompt}  # 프롬프트 연결
)

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7),
    max_tokens=prompt.config.get("max_tokens", 1000)
)

# 체인 생성
chain = langchain_prompt | model

# 실행 (추적과 함께)
response = chain.invoke(
    {"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}
)

print(f"🎬 AI 영화 평론: {response.content}")

#### 채팅 프롬프트와 LangChain 연동

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅 프롬프트 가져오기
chat_prompt = langfuse.get_prompt("movie-critic-chat", label="staging")

# ChatPromptTemplate 생성
langchain_chat_prompt = ChatPromptTemplate.from_messages(
    chat_prompt.get_langchain_prompt(type="chat")
)

# 메타데이터 추가 (프롬프트 연결)
langchain_chat_prompt.metadata = {"langfuse_prompt": chat_prompt}

# 모델 설정
model = ChatOpenAI(
    model=chat_prompt.config.get("model", "gpt-4.1-mini"),
    temperature=chat_prompt.config.get("temperature", 0.7)
)

# 체인 실행
chain = langchain_chat_prompt | model
response = chain.invoke(
    {"criticLevel": "영화 애호가", "movie": "기생충"},
    config={"callbacks": [langfuse_handler]}
)

print(f"🎬 AI 영화 평론 (채팅): {response.content}")


---

## 6. 실습 문제

### 🎯 기초 실습

#### 실습 1: 프롬프트 생성하기

다음 요구사항에 맞는 프롬프트를 생성하세요:

In [ ]:
# TODO: 다음 조건으로 프롬프트 생성
# - 이름: "recipe-generator"
# - 타입: "text"
# - 내용: "{{cuisine}} 요리 전문가로서, {{ingredients}} 재료를 사용한 {{difficulty}} 난이도의 요리 레시피를 제공해주세요."
# - 라벨: ["development"]
# - 태그: ["cooking", "recipe", "food"]
# - 설정: model="gpt-4.1-mini", temperature=0.8

# 여기에 코드를 작성하세요

#### 실습 2: 채팅 프롬프트 생성 및 활용

In [ ]:
# TODO: 다음 조건으로 채팅 프롬프트 생성
# - 이름: "language-tutor"
# - 타입: "chat"
# - 시스템 메시지: "당신은 {{language}} 언어 전문 튜터입니다. 학습자의 {{level}} 수준에 맞춰 친근하고 이해하기 쉽게 설명해주세요."
# - 사용자 메시지: "{{topic}}에 대해 설명해주세요."
# - 라벨: ["staging"]
# - 설정: model="gpt-4.1", temperature=0.6

# 여기에 코드를 작성하세요

In [ ]:
# TODO: 생성한 프롬프트를 사용하여 LangChain과 연동하고 실행하기
# 입력값: language="영어", level="초급", topic="현재 완료 시제"

# 여기에 코드를 작성하세요

### 🚀 심화 실습

#### 실습 3: 버전 관리 및 A/B 테스트

In [ ]:
# TODO: 영화 추천 시스템 구현
# 1. "movie-recommender" 프롬프트 생성 (버전 A)
# 2. 개선된 버전 생성 (버전 B)
# 3. 두 버전으로 같은 입력에 대해 실행하고 결과 비교
# 4. 성능이 좋은 버전을 프로덕션으로 승격

# 여기에 코드를 작성하세요

---

## 📝 주요 포인트

1. **최신 SDK 사용**: v3 SDK (OpenTelemetry 기반) 권장
2. **자동 캐싱**: 60초 TTL로 성능 최적화
3. **버전 관리**: 라벨을 통한 환경별 배포
4. **LangChain 통합**: 네이티브 CallbackHandler 지원

---

In [ ]:
# 기초 실습 1 예시 답안

recipe_prompt = langfuse.create_prompt(
    name="recipe-generator",
    type="text",
    prompt="{{cuisine}} 요리 전문가로서, {{ingredients}} 재료를 사용한 {{difficulty}} 난이도의 요리 레시피를 제공해주세요.",
    labels=["development"],
    tags=["cooking", "recipe", "food"],
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.8
    }
)
print(f"✅ 레시피 프롬프트 생성: {recipe_prompt.name}")

In [ ]:
# 기초 실습 2 예시 답안

tutor_prompt = langfuse.create_prompt(
    name="language-tutor",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{language}} 언어 전문 튜터입니다. 학습자의 {{level}} 수준에 맞춰 친근하고 이해하기 쉽게 설명해주세요."
        },
        {
            "role": "user",
            "content": "{{topic}}에 대해 설명해주세요."
        }
    ],
    labels=["staging"],
    config={
        "model": "gpt-4.1",
        "temperature": 0.6
    }
)

In [ ]:
# 기초 실습 2 예시 답안

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 가져오기 및 변환
retrieved_prompt = langfuse.get_prompt("language-tutor", label="staging")
chat_template = ChatPromptTemplate.from_messages(
    retrieved_prompt.get_langchain_prompt(type="chat")
)
chat_template.metadata = {"langfuse_prompt": retrieved_prompt}

# 모델 및 체인 구성
model = ChatOpenAI(
    model=retrieved_prompt.config.get("model", "gpt-4.1"),
    temperature=retrieved_prompt.config.get("temperature", 0.6)
)

chain = chat_template | model

# 실행
response = chain.invoke(
    {
        "language": "영어",
        "level": "초급",
        "topic": "현재 완료 시제"
    },
    config={"callbacks": [langfuse_handler]}
)

print(f"📚 언어 튜터 응답: {response.content}")

In [ ]:
# 심화 실습 3 예시 답안

# 버전 A 생성
version_a = langfuse.create_prompt(
    name="movie-recommender",
    type="text",
    prompt="{{genre}} 장르의 영화를 {{count}}개 추천해주세요.",
    labels=["experiment-a"],
    tags=["recommendation", "movie", "v1"],
    config={"model": "gpt-4.1-mini", "temperature": 0.7}
)

# 버전 B 생성 (개선된 버전)
version_b = langfuse.create_prompt(
    name="movie-recommender",
    type="text",
    prompt="{{genre}} 장르의 영화를 {{count}}개 추천해주세요. 각 영화에 대해 간단한 줄거리와 추천 이유를 포함해주세요.",
    labels=["experiment-b"],
    tags=["recommendation", "movie", "v2"],
    config={"model": "gpt-4.1-mini", "temperature": 0.7}
)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def test_prompt_version(label: str, test_input: dict):
    """프롬프트 버전 테스트"""
    prompt = langfuse.get_prompt("movie-recommender", label=label)
    
    template = PromptTemplate.from_template(
        prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": prompt}
    )
    
    model = ChatOpenAI(
        model=prompt.config.get("model"),
        temperature=prompt.config.get("temperature")
    )
    
    chain = template | model
    response = chain.invoke(
        test_input,
        config={"callbacks": [langfuse_handler]}
    )
    
    return response.content

# A/B 테스트 실행
test_input = {"genre": "SF", "count": "3"}

print("🅰️ 버전 A 결과:")
result_a = test_prompt_version("experiment-a", test_input)
print(result_a)
print("\n" + "="*50 + "\n")

print("🅱️ 버전 B 결과:")
result_b = test_prompt_version("experiment-b", test_input)
print(result_b)

In [ ]:
# 더 나은 버전을 프로덕션으로 승격 (여기서는 B 선택)
production_version = langfuse.create_prompt(
    name="movie-recommender",
    type="text",
    prompt="{{genre}} 장르의 영화를 {{count}}개 추천해주세요. 각 영화에 대해 간단한 줄거리와 추천 이유를 포함해주세요.",
    labels=["production"],   # 프로덕션 라벨 지정 
    tags=["recommendation", "movie", "final"],
    config={"model": "gpt-4.1-mini", "temperature": 0.7}
)

print(f"🎯 프로덕션 버전 배포 완료: v{production_version.version}")

---